In [1]:
!RNAdistance --version
#RNAdistance 2.3.0cuda

RNAdistance 2.3.0cuda


In [2]:
import os
import subprocess
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
DATA_PATH = "./data/"

In [4]:
raw_fn = "archiveII_220808"
dref_all = pd.read_csv(DATA_PATH + raw_fn + ".csv", index_col = 1)
display(dref_all.head(3))
display(dref_all.shape)

,Unnamed: 0,method,sequence,structure,motif,base_pairs,header,date,resolution,len
id,,,,,,,,,,
5s_Acholeplasma-laidlawii-1,0,NaN,UCUGGUGACGAUAGGUAAGAUGGUUCACCUGUUCCCAUCCCGAACA...,((((((((......((((((((....((((((.............)...,NaN,"[[1, 111], [2, 110], [3, 109], [4, 108], [5, 1...",NaN,NaN,NaN,112
5s_Acidovorax-temperans-1,1,NaN,UGCCUGAUGACCAUAGCAAGUUGGUACCACUCCUUCCCAUCCCGAA...,.(((((((((.....((((((((.....((((((...............,NaN,"[[2, 115], [3, 114], [4, 113], [5, 112], [6, 1...",NaN,NaN,NaN,115
tmRNA_Stre.gord._TRW-29390_1-349,2,NaN,GGGGUCGUUACGGAUUCGACAGGCAUUAUGAGGCAUAUUUUGCGAC...,(((((((............((((((((....(((((((((..((((...,NaN,"[[1, 345], [2, 344], [3, 343], [4, 342], [5, 3...",NaN,NaN,NaN,349


(3975, 10)

In [5]:
#dref = dref_all.head(10) # testing
dref = dref_all

Results = pd.DataFrame(np.zeros((len(dref.index),len(dref.index))))-1
Results.index = dref.index
Results.columns = dref.index
for i,rowi in tqdm(dref.iterrows(),total=len(dref)):
    for j,rowj in dref.iterrows():
        if Results.loc[j,i] < 0:
            echo_line = rowi.structure + "\n" + rowj.structure
            # RNAdistance cannot handle pseudoknots (treated as unpaired) https://pubmed.ncbi.nlm.nih.gov/36077037/
            echo_line = echo_line.replace('<','.').replace('>','.').replace('{','.').replace('}','.')
            oo = subprocess.check_output(["RNAdistance"], input=echo_line.encode('utf-8'))

            if oo != b"":
                max_len = len(rowi.structure) if len(rowi.structure)>len(rowj.structure) else len(rowj.structure) 
                Results.loc[i,j] = float(oo[2:])/max_len
            else:
                Results.loc[i,j] = np.nan
                print("Warning: RNAdistance error with {} and {}".format(i, j))
        else:
            Results.loc[i,j] = Results.loc[j,i]

100%|██████████| 3975/3975 [18:08:28<00:00, 16.43s/it]   


In [8]:
Results.to_csv("data/rnadist_f_all.csv")

In [7]:
Results.to_hdf("data/rnadist_f_all.h5", key='rnadist', mode='w')